#  Weekly project

## 1. Scraping the first page

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup
def scrape_tiki(url = 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?'):
    soup = get_url(url)
    items = soup.find_all('div',{"class": "product-item"})
    data = []
    for item in items:
        try: 
            dic = {"product_id":"","seller_id":"","title":"","price":"","image_url":""}
            dic["product_id"] = item["data-id"]
            dic["seller_id"] = item["data-seller-product-id"]
            dic["title"] = item["data-title"]
            dic["price"] = item["data-price"]
            if item.find("span",{"class":"image"}):
                dic["image_url"] = item.find("span",{"class":"image"}).img["src"]
            data.append(dic)
        except:
            print("We got an error")
    
    return (data)

In [ ]:
data = scrape_tiki()
data

In [ ]:
page1 = pd.DataFrame(data,columns = data[0].keys())

In [ ]:
page1.head()

## 2. Scraping all pages

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint

def get_url(url):  # create a function to parse the url
    """Get parsed HTML from url
      Input: url to the webpage
      Output: Parsed HTML text of the webpage
    """
    # Send GET request
    r = requests.get(url)  
    
    # Parse HTML text
    soup = BeautifulSoup(r.text, 'html.parser') 
    return soup

def scrape_tiki_all(url = "https://tiki.vn/dien-thoai-may-tinh-bang/c1789?"):
    """Scrape listing info of phone cat of tiki.vn
      Input: url of a cat. Default: https://tiki.vn/dien-thoai-may-tinh-bang/c1789?
      Output: A list containing scraped data of listing items
    """
    url_base = url
    
    # List containing data of all articles
    data = []
    
    page = 1
    
    items = True
    
    # start to find target tags and extract the item infos, stop when no info of items is found
    while items != []:
        
        # Get parsed HTML
        soup = get_url(url)
        
        # Find all tags that contain required info
        items = soup.find_all('div',{"class": "product-item"}) 
        
        # Extract information of each tag
        for item in items: 
            
            # We use the try-except blocks to handle errors
            try: 
                
                # Each tag is dictionary containing the required information
                dic = {"product_id":"","seller_id":"","title":"","price":"","image_url":""}
                dic["product_id"] = item["data-id"]
                dic["seller_id"] = item["data-seller-product-id"]
                dic["title"] = item["data-title"]
                dic["price"] = item["data-price"]
                
                # There are some articles without img tag...
                if item.find("span",{"class":"image"}):
                    dic["image_url"] = item.find("span",{"class":"image"}).img["src"]
                
                # Append the dictionary to data list
                data.append(dic)
            except:
                
                 # Skip if error and print error message
                print("We got an error")
        
        # print out the page number and items to keep track
        print(page, len(data))
        
        # increment page
        page += 1
        
        # create the url of the next page
        url = url_base + "&page=" +str(page)
        
        #  control the scraping speed
        time.sleep(randint(1,4))
    
    
    return data

In [ ]:
# Test the scraper
phone = scrape_tiki_all()
phone

In [ ]:
# Save data to a DataFrame
phone_product = pd.DataFrame(phone,columns = phone[0].keys())
phone_product

In [ ]:
# extract to pickle
phone_product.to_pickle("./phone_product.pkl")

In [ ]:
# extract to csv
phone_product.to_csv("./phone_product.csv", index = False)